In [1]:
import zipfile
import os

def unzip_file(zip_file_path, extract_path):
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

zip_file_path = '/content/drive/MyDrive/Data/final_dataset.zip'
extract_path = 'extracted_files'

unzip_file(zip_file_path, extract_path)

In [12]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def visualize_random_image_with_bbox(image_dir, label_dir, class_names):
    image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not image_files:
        print(f"No images found in {image_dir}")
        return

    random_image_file = random.choice(image_files)
    image_path = os.path.join(image_dir, random_image_file)

    label_file_name = os.path.splitext(random_image_file)[0] + '.txt'
    label_path = os.path.join(label_dir, label_file_name)

    if not os.path.exists(label_path):
        print(f"No label found for {random_image_file}")
        return

    try:
        img = Image.open(image_path)
        img_width, img_height = img.size

        fig, ax = plt.subplots(1)
        ax.imshow(img)

        with open(label_path, 'r') as f:
            for line in f:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                x_center_pixel = x_center * img_width
                y_center_pixel = y_center * img_height
                width_pixel = width * img_width
                height_pixel = height * img_height

                x_min = x_center_pixel - (width_pixel / 2)
                y_min = y_center_pixel - (height_pixel / 2)


                rect = patches.Rectangle((x_min, y_min), width_pixel, height_pixel, linewidth=1, edgecolor='r', facecolor='none')
                ax.add_patch(rect)


                class_name = class_names[int(class_id)] if int(class_id) < len(class_names) else f"Class {int(class_id)}"
                ax.text(x_min, y_min - 5, class_name, color='red', fontsize=8, backgroundcolor='white')

        plt.title(f"Image: {random_image_file}")
        plt.show()
    except Exception as e:
        print(f"Error processing {random_image_file}: {e}")

class_names = ["Face", "Lips", "Glasses"]

image_directory = "/content/extracted_files/train/images"
label_directory = "/content/extracted_files/train/labels"

visualize_random_image_with_bbox(image_directory, label_directory, class_names)


In [3]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/112.6 GB disk)


In [4]:
import torch
from ultralytics import YOLO

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_train = YOLO("yolov8n.pt").to(device)

100%|██████████| 6.25M/6.25M [00:00<00:00, 130MB/s]


In [5]:
#model_train.tune(data="/content/extracted_files/data.yaml", epochs=50, iterations=300, optimizer="AdamW", plots=False, save=False, val=False)

In [6]:
data_yaml_path = "/content/extracted_files/data.yaml"
model_train.train(
    data=data_yaml_path,
    epochs=100,
    imgsz=640,
    device=device
    )

engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/extracted_files/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, 

100%|██████████| 755k/755k [00:00<00:00, 25.5MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 107MB/s]


AMP: checks passed ✅


...

                   all          5         15      0.929      0.996      0.995       0.63



100 epochs completed in 0.046 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 12.53it/s]


                   all          5         15       0.94      0.922      0.995      0.665
                  face          5          5      0.946          1      0.995      0.712
                  lips          5          5          1      0.765      0.995      0.557
            spectacles          5          5      0.874          1      0.995      0.726
Speed: 0.2ms preprocess, 3.9ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ccad72d8c10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [10]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def predict_and_visualize(image_path, model):
    img = Image.open(image_path)
    results = model.predict(source=image_path, conf=0.25, save=False, save_txt=False)

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    axes[0].imshow(img)
    axes[0].set_title("Original Image")
    axes[0].axis('off')

    ax = axes[1]
    ax.imshow(img)

    for result in results[0].boxes:
        x_min, y_min, x_max, y_max = result.xyxy[0].tolist()
        conf = result.conf.item()
        class_id = int(result.cls.item())
        class_name = model.names[class_id]
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x_min, y_min - 5, f"{class_name} {conf:.2f}", color='red', fontsize=8, backgroundcolor='white')

    ax.set_title("Predicted Image")
    ax.axis('off')
    plt.tight_layout()
    plt.show()

model_path = "/content/runs/detect/train/weights/best.pt"
model = YOLO(model_path)

uploaded_image_path = "/content/istockphoto-1134003578-612x612.jpg"
predict_and_visualize(uploaded_image_path, model)



image 1/1 /content/istockphoto-1134003578-612x612.jpg: 448x640 1 face, 1 lips, 1 spectacles, 126.1ms
Speed: 16.8ms preprocess, 126.1ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 640)
